In [1]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

In [22]:
scenario = 'rfr'
if scenario[-1] == 'r':
    pscore = 'r2'
else:
    pscore = 'auc'

In [23]:
kf5 = pd.read_csv(f'evaluation_{scenario}.csv')
kf10 = pd.read_csv(f'evaluation_{scenario}_10.csv')

In [24]:
kf5['n_splits'] = 5
kf10['n_splits'] = 10

In [25]:
data = pd.concat([kf5, kf10])

In [26]:
grouped = data.groupby(['name', 'mode', 'n_splits']).apply(lambda pdf: pd.Series({'scores': pdf.sort_values('fold')[pscore].values.tolist()})).reset_index(drop=False)

In [27]:
def process(pdf):
    leq5 = pdf[(pdf['mode'] == '<=') & (pdf['n_splits'] == 5)].iloc[0]
    leq10 = pdf[(pdf['mode'] == '<=') & (pdf['n_splits'] == 10)].iloc[0]
    l5 = pdf[(pdf['mode'] == '<') & (pdf['n_splits'] == 5)].iloc[0]
    l10 = pdf[(pdf['mode'] == '<') & (pdf['n_splits'] == 10)].iloc[0]

    return pd.Series({
        '5_l': wilcoxon(leq5['scores'], l5['scores'], alternative='less', zero_method='zsplit').pvalue < 0.05,
        '5_g': wilcoxon(leq5['scores'], l5['scores'], alternative='greater', zero_method='zsplit').pvalue < 0.05,
        '10_l': wilcoxon(leq10['scores'], l10['scores'], alternative='less', zero_method='zsplit').pvalue < 0.05,
        '10_g': wilcoxon(leq10['scores'], l10['scores'], alternative='greater', zero_method='zsplit').pvalue < 0.05
        })

In [28]:
grouped.groupby('name').apply(process)

,5_l,5_g,10_l,10_g
name,,,,
airfoil,True,False,True,False
autoMPG6,False,False,False,False
baseball,False,True,False,True
cpu_performance,False,True,False,True
daily-demand,True,False,True,False
diabetes,False,True,False,True
excitation_current,False,True,False,True
laser,False,True,False,True
maternal_health_risk,True,False,True,False
